In [ ]:
import pandas as pd
import numpy as np
from qiskit import QuantumCircuit
def apply_angle_encoding(
    qc: QuantumCircuit,
    thetas: np.ndarray,
    axes: Sequence[str],
) -> None:
    """
    Apply angle encoding per feature/qubit.

    Each feature angle θ_i is applied using the sequence of rotations
    specified in `axes` (e.g., ("ry","rz") applies RY(θ_i) then RZ(θ_i)).
    """
    valid = {"rx", "ry", "rz"}
    axes = tuple(ax.lower() for ax in axes)
    if not axes:
        raise ValueError("encoding_axes must contain at least one of {'rx','ry','rz'}.")
    if any(ax not in valid for ax in axes):
        raise ValueError(f"encoding_axes must be in {valid}, got {axes}.")

    for q, theta in enumerate(thetas):
        for ax in axes:
            if ax == "rx":
                qc.rx(theta, q)
            elif ax == "ry":
                qc.ry(theta, q)
            elif ax == "rz":
                qc.rz(theta, q)

data = pd.read_csv('../Data/X_train_scaled.csv')